### LIBRARIES

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import netCDF4
import gzip
from shutil import copyfile
import matplotlib.pyplot as plt
import shapely.wkt
from shapely.geometry import shape
import math
import warnings
from pathlib import Path
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, MultiLineString
from pathlib import Path
warnings.filterwarnings("ignore")

### ENV VARIABLES

In [2]:
PATH = '/home/helvecioneto/01_IARA/RADAR/iara_beta_v6/output/S201409070000_E201409100000_VDBZc_T20_L5_SPLTTrue_MERGTrue_TCORTrue_PCORFalse.zip'
DATA_PATH = '/home/helvecioneto/SINAPSE_01/DADOS/sbandradar/'
VAR_NAME = 'DBZc'
LEVEL = 5
THRESHOLD = [20,35,40]
OUTPUT = '../output/'
NC_OUTPUT = '../output/data/'
# CLUSTERS = ''

In [3]:
### READ TRACK FROM PATH
def read_track(path):
    zip_file = ZipFile(path)
    for f in zip_file.infolist():
        if f.filename.endswith('.csv') and 'FINAL' in f.filename:
            tracking_df = pd.read_csv(zip_file.open(f))
    return tracking_df

In [4]:
### OPEN NETCDF
def open_file(file_path):
    try:
        with gzip.open(file_path) as gz:
            with netCDF4.Dataset('dummy', mode='r', memory=gz.read()) as nc:
                data = nc.variables[VAR_NAME][0][LEVEL][:].filled()
                data[data == -9999.] = np.NAN
        return data
    except:
        pass

In [5]:
### FAMILY GENERATOR
def create_fam(df):
    ## REORGANIZE FAM BY MULTINDEX
    df = df.groupby('uid').apply(lambda x: x.sort_values(["uid"], ascending = False))
    index_list = df.index.tolist()
    new_list = []
    for i in index_list:
        new_list.append(('Fam_'+str(i[0]),i[1]))
    mux = pd.MultiIndex.from_tuples(new_list)
    df.index = mux
    return df

In [6]:
### GET LAT LONG MATRIX
def get_latlong_matrix(file_path):
    with gzip.open(file_path) as gz:
        with netCDF4.Dataset('dummy', mode='r', memory=gz.read()) as nc:
            data_var = nc.variables.keys()
            for k in data_var:
                if k.lower().startswith('lat'):
                    llat = k
                elif k.lower().startswith('lon'):
                    llon = k
            lon = nc.variables[llon][:]
            lat = nc.variables[llat][:]
    return lon,lat

In [7]:
### GENERATE PATCH FILES
def generate_path_files(df,DATA_PATH):
    for i,row in df.iterrows():
        file = str(pd.to_datetime(row.timestamp).strftime(DATA_PATH+'%Y/%m/sbmn_cappi_%Y%m%d_%H%M.nc.gz'))
        df.loc[i,'nc_file'] = file
        cluster = pd.to_datetime(row.timestamp).strftime('clusters/%Y%m%d_%H%M%S_clu.npz')
        df.loc[i,'cluster_file'] = cluster
    return df

In [8]:
## OPEN CLUSTERS BY DBSCAN
def open_cluster(path,file_path):
    try:
        zip_file = ZipFile(path)
        cluster = np.load(zip_file.open(file_path))['arr_0']
        return cluster
    except:
        pass

In [9]:
## Calculate angle of Linestring
def angle(x):
    p1 = list(shape(x).coords)[0]
    p2 = list(shape(x).coords)[1]

    rad_ = math.atan2(p2[1]-p1[1], p2[0]-p1[0])
    direction = math.degrees(rad_)

    if direction < 0:
        direction = direction + 360
    return direction

In [10]:
# timestamp
# uid     -> identificador unico
# time    -> timeindex

# lat lon -> centroid

# size_20    -> tamanho do sistema
# total_size_30    -> tamanho do sistema
# total_size_40    -> tamanho do sistema

# dsize   -> Delta área

# mean_ref_20 -> 
# mean_total_ref_30 -> 
# mean_total_ref_40 ->

# angle_20 ->
# avg_angle_30 ->
# avg_angle_40 ->

# vel_20_orig ->
# vel_20_cor ->

# avg_vel_30 ->
# avg_vel_40 ->

# n_cluster_30 ->
# n_cluster_40 ->

# status -> 
# lifetime -> delta_t


# MAX_REFLECT -> Máxima refletivida do cluster no tempo t
# DMAX_REFLECT -> Diferença entre a max (t-1) com (t)
# DMEAN_REFLECT -> Média

#### DÚVIDA, PODE CALCULAR DIRETO?

#### UM NOTEBOOK PARA CONVERTER TRACKING PARA FAMILY (No mínimo 2 clusters por familia)
#### UM NOTEBOOK PARA VISUALIZAR FAMILIY E CONSULTAR CLUSTERS

In [11]:
def calc_dbz(df,threshold,path):
    for i, row in df.iterrows():
        ### GET REFLECT AND PIXEL SIZES
        cluster_matrix = open_cluster(path,row.cluster_file)
        dbz_matrix = open_file(row.nc_file)
        if cluster_matrix is not None and dbz_matrix is not None:
            for t in range(len(threshold)):
                clt_matrix = cluster_matrix[:,:,t]
                x,y = np.where(clt_matrix == row.id_t)
                dbz_list = dbz_matrix[x,y]
                if len(dbz_list) > 0:
                    ### MEAN REFLECT
                    mmh6 = np.mean(10**(dbz_list/10))
                    dbz_mean = (10*np.log10(mmh6))
                    dbz_max = np.max(dbz_list)

                    if t == 0:
                        df.loc[i,'mean_ref_'+str(threshold[t])] = dbz_mean
                        df.loc[i,'max_ref_'+str(threshold[t])] = dbz_max
                        df.loc[i,'size_'+str(threshold[t])] = len(x)
                    else:
                        df.loc[i,'mean_total_ref_'+str(threshold[t])] = dbz_mean
                        df.loc[i,'total_size_'+str(threshold[t])] = int(len(x))
        else:
            del cluster_matrix,dbz_matrix
        
        ### GET ORIG ANGLE AND VELM
        if row.linestring != '-1':
            for t in range(len(threshold)):
                if t == 0:
                    df.loc[i,'angle_'+str(threshold[t])+'_cor'] = row.angle
                    df.loc[i,'angle_'+str(threshold[t])+'_orig'] = angle(shapely.wkt.loads(row.linestring))
                    
                    ## VELM
                    df.loc[i,'vel_'+str(threshold[t])+'_orig'] = (row['length'] * 2) / 0.2
                    df.loc[i,'vel_'+str(threshold[t])+'_cor'] = row.velm
                else:
                    df.loc[i,'avg_angle_'+str(threshold[t])] = row['internal_angle_mean_'+str(threshold[t])]
                    df.loc[i,'avg_vel_'+str(threshold[t])] = row['internal_velm_'+str(threshold[t])]
                    
        ### GET LON LAT
        if row.centroid_t != '-1':
            point = list(shape(shapely.wkt.loads(row.centroid_t)).coords)[0]
            p0,p1 = int(np.round(point[0])),int(np.round(point[1]))
            long = LON[p0][p1]
            latg = LAT[p0][p1]
            
            df.loc[i,'lat'] = latg
            df.loc[i,'lon'] = long
            df.loc[i,'p0'] = p0
            df.loc[i,'p1'] = p1  
            
        ### GET NUMBER OF CLUSTERS
        for t in threshold[1:]:
            if row['n_poly_'+str(t)] == -1:
                df.loc[i,'n_cluster_'+str(t)] = 0
            else:
                df.loc[i,'n_cluster_'+str(t)] = row['n_poly_'+str(t)]
    return df

In [12]:
## TRACK DATA FRAME
track_df = read_track(PATH)
track_df = generate_path_files(track_df,DATA_PATH)
LON,LAT = get_latlong_matrix(track_df.iloc[0]['nc_file'])

to_fam_df = calc_dbz(track_df,THRESHOLD,PATH)

In [14]:
used_columns = ['timestamp','time','uid','id_t','lat','lon','p0','p1']
for t in range(len(THRESHOLD)):
    if t == 0:
        used_columns.append('size_'+str(THRESHOLD[t]))
        used_columns.append('mean_ref_'+str(THRESHOLD[t]))
        used_columns.append('max_ref_'+str(THRESHOLD[t]))
        used_columns.append('angle_'+str(THRESHOLD[t])+'_orig')
        used_columns.append('angle_'+str(THRESHOLD[t])+'_cor')
        used_columns.append('vel_'+str(THRESHOLD[t])+'_orig')
        used_columns.append('vel_'+str(THRESHOLD[t])+'_cor')
    else:
        used_columns.append('mean_total_ref_'+str(THRESHOLD[t]))
        used_columns.append('total_size_'+str(THRESHOLD[t]))
        used_columns.append('n_cluster_'+str(THRESHOLD[t]))
        used_columns.append('avg_angle_'+str(THRESHOLD[t]))
        used_columns.append('avg_vel_'+str(THRESHOLD[t]))
        
used_columns.append('status')
used_columns.append('delta_t')
used_columns.append('nc_file')
used_columns.append('cluster_file')

used_columns.append('line_'+str(THRESHOLD[0]))
used_columns.append('line_'+str(THRESHOLD[1]))
used_columns.append('line_'+str(THRESHOLD[2]))

used_columns.append('geom_intersect')
used_columns.append('geom_'+str(THRESHOLD[0]))
used_columns.append('geom_'+str(THRESHOLD[1]))
used_columns.append('geom_'+str(THRESHOLD[2]))
to_fam_df['delta_t'] = to_fam_df['lifetime']

## GEOMETRIES
to_fam_df['geom_intersect'] = to_fam_df['intersect_geom']
to_fam_df['geom_'+str(THRESHOLD[0])] = to_fam_df['geometry_t']
to_fam_df['geom_'+str(THRESHOLD[1])] = to_fam_df['geom_'+str(THRESHOLD[1])]
to_fam_df['geom_'+str(THRESHOLD[2])] = to_fam_df['geom_'+str(THRESHOLD[2])]


to_fam_df['line_'+str(THRESHOLD[0])] = to_fam_df['linestring']
to_fam_df['line_'+str(THRESHOLD[1])] = to_fam_df['internal_linestring_'+str(THRESHOLD[1])]
to_fam_df['line_'+str(THRESHOLD[2])] = to_fam_df['internal_linestring_'+str(THRESHOLD[2])]
        
to_fam_df = to_fam_df[used_columns]

In [15]:
def redefine(df):
    df_copy = df
    #### REPLACE 0 to NAN
    df_copy = df_copy[df_copy.columns[6:]].replace(0,np.nan)
    #### REPLACE -1 to NAN
    df_copy = df_copy.replace(-1,np.nan)
    #### REPLACE '-1' to NAN
    df_copy = df_copy.replace('-1',np.nan)
    df_final = pd.concat([df[df.columns[:6]],df_copy],axis=1)
    return df_final

In [16]:
def translate_geometries(df_c,geo_columns,lat,lon):
    df_t = df_c.copy()
    geometry = None
    for g in geo_columns:
        for i in range(len(df_t[g])):
            if type(df_t[g][i]) == str and not df_t[g][i].startswith('-1') and not df_t[g][i].startswith('0') and not df_t[g][i].startswith('-1.0'):
                try:
                    geo = shapely.wkt.loads(df_t[g][i])
                    if type(geo) == Polygon:
                        geometry = trans_poly(geo,lat,lon)
                        df_t.loc[i,g] = geometry.wkt
                    elif type(geo) == MultiPolygon:
                        geometry = trans_multipoly(geo,lat,lon)
                        df_t.loc[i,g] = geometry.wkt
                    elif type(geo) == LineString:
                        geometry = trans_lines(geo,lat,lon)
                        df_t.loc[i,g] = geometry.wkt
                    elif type(geo) == MultiLineString:
                        geometry = trans_multilines(geo,lat,lon)
                        df_t.loc[i,g] = geometry.wkt
                    else:
                        geometry = None
                        geo = None
                except Exception as e:
                    geometry = None
                    df_t.loc[i,g] = df_t[g][i]
            else:
                df_t.loc[i,g] = None
    return df_t

def trans_poly(geom,latitude,longitude):
    x,y = geom.exterior.coords.xy
    points = []
    for v in range(len(x)):
        points.append([longitude[int(np.round(y[v]))][int(np.round(x[v]))],latitude[int(np.round(y[v]))][int(np.round(x[v]))]])
    poly = Polygon(list(zip(np.array(points)[:,0],np.array(points)[:,1])))
    return poly

def trans_multipoly(geom,latitude,longitude):
    polys2 = []
    for gg in geom:
        x,y = gg.exterior.coords.xy
        points = []
        for v in range(len(x)):
            points.append([longitude[int(np.round(y[v]))][int(np.round(x[v]))],latitude[int(np.round(y[v]))][int(np.round(x[v]))]])
        polys2.append(Polygon(list(zip(np.array(points)[:,0],np.array(points)[:,1]))))
    return MultiPolygon(polys2)

def trans_lines(geom,latitude,longitude):
    x,y = geom.xy
    p = []
    for v in range(len(x)):
        p.append(Point([longitude[int(np.round(y[v]))][int(np.round(x[v]))],latitude[int(np.round(y[v]))][int(np.round(x[v]))]]))
    return LineString(p)

def trans_multilines(geom,latitude,longitude):
    lines2 = []
    for gg in geom:
        x,y = gg.xy
        p = []
        for v in range(len(x)):
            p.append(Point([longitude[int(np.round(y[v]))][int(np.round(x[v]))],latitude[int(np.round(y[v]))][int(np.round(x[v]))]]))
        lines2.append(LineString(p))
    return MultiLineString(lines2)

In [17]:
### CREATE DIRs
Path(OUTPUT).mkdir(parents=True, exist_ok=True)
Path(OUTPUT+'clusters').mkdir(parents=True, exist_ok=True)
Path(OUTPUT+'data').mkdir(parents=True, exist_ok=True)

In [18]:
### EXTRACT CLUSTERS
with ZipFile(PATH, 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if 'clusters/' in fileName:
            zipObj.extract(fileName, OUTPUT)

In [19]:
#### COPY NC_FILES
for i,row in to_fam_df.iterrows():
#     print(str(NC_OUTPUT+Path(row.nc_file).name))
#     print(OUTPUT+row.cluster_file)
#     print('')
#     print(OUTPUT[7:]+row.cluster_file,OUTPUT[7:]+'data/'+row.nc_file[27:])
#     print(row.nc_file,OUTPUT[7:]+'../fam/data/'+row.nc_file[27:])
    try:
        copyfile(row.nc_file,str(NC_OUTPUT+Path(row.nc_file).name))
        to_fam_df.loc[i,'nc_file'] = str(NC_OUTPUT+Path(row.nc_file).name)
        to_fam_df.loc[i,'cluster_file'] = OUTPUT+row.cluster_file
    except:
        pass

In [20]:
geo_cols = []
for c in to_fam_df.columns:
    if 'geom_' in c or 'line' in c:
        geo_cols.append(c)

In [21]:
to_fam_df = redefine(to_fam_df)

In [24]:
## TRANSLATE GEOMETRIES
to_fam_df = translate_geometries(to_fam_df,geo_cols,LAT,LON)

In [25]:
fam_df = create_fam(to_fam_df)

In [26]:
### FILTER BY SIZE
sized_fams = []
for i,g in fam_df.groupby(level=0):
    if len(g) >= 2:
        sized_fams.append(i)

In [27]:
### FINAL FRAME
fam_df = fam_df.loc[sized_fams]

In [28]:
### CALCULATE DIFFERENCES
for i,g in fam_df.groupby(level=0):
    for t in range(len(THRESHOLD)):
        if t == 0:
            fam_df.loc[i,'dsize_'+str(THRESHOLD[t])] = g['size_'+str(THRESHOLD[t])].diff()
            fam_df.loc[i,'dmean_ref_'+str(THRESHOLD[t])] = g['mean_ref_'+str(THRESHOLD[t])].diff()
        else:
            fam_df.loc[i,'dtotal_size_'+str(THRESHOLD[t])] = g['total_size_'+str(THRESHOLD[t])].diff()
            fam_df.loc[i,'dmean_total_ref_'+str(THRESHOLD[t])] = g['mean_total_ref_'+str(THRESHOLD[t])].diff()

In [29]:
fam_df

timestamp  time  uid  id_t       lat        lon     p0  \
Fam_0  0    2014-09-07 00:00:00     0    0     2 -1.423926 -60.710175  216.0   
       10   2014-09-07 00:12:00     1    0     3 -1.495779 -60.728176  212.0   
Fam_1  1    2014-09-07 00:00:00     0    1     5 -2.358212 -60.602760  164.0   
       11   2014-09-07 00:12:00     1    1     4 -2.412090 -60.638752  161.0   
Fam_10 20   2014-09-07 00:24:00     2   10     8 -1.675547 -60.512596  202.0   
...                         ...   ...  ...   ...       ...        ...    ...   
Fam_97 533  2014-09-07 19:00:00    95   97    20 -3.094856 -60.531185  123.0   
       548  2014-09-07 19:12:00    96   97    15 -3.130749 -60.603176  121.0   
       566  2014-09-07 19:24:00    97   97    21 -3.202613 -60.603218  117.0   
Fam_98 538  2014-09-07 19:00:00    95   98    44 -3.130779 -59.433620  121.0   
       554  2014-09-07 19:12:00    96   98    39 -3.148715 -59.379631  120.0   

               p1  size_20  mean_ref_20  ...  \
Fam_0  0     80.0     66.0    35.540233  ...   
       10    79.0     34.0    26.850009  ...   
Fam_1  1     86.0     75.0    39.058735  ...   
       11    84.0     41.0    37.308171  ...   
Fam_10 20    91.0     82.0    38.440909  ...   
...           ...      ...          ...  ...   
Fam_97 533   90.0    169.0    40.267625  ...   
       548   86.0    215.0    37.533064  ...   
       566   86.0    160.0    37.018101  ...   
Fam_98 538  151.0    209.0    40.025382  ...   
       554  154.0    238.0    39.563215  ...   

                                               geom_intersect  \
Fam_0  0                                                 None   
       10   MULTIPOLYGON (((-58.28065872192383 -3.91993093...   
Fam_1  1                                                 None   
       11   POLYGON ((-59.25323104858398 -3.74150490760803...   
Fam_10 20   MULTIPOLYGON (((-58.42523956298828 -3.61478638...   
...                                                       ...   
Fam_97 533  POLYGON ((-59.86537933349609 -3.74177384376525...   
       548  MULTIPOLYGON (((-59.97340774536133 -3.50821971...   
       566  MULTIPOLYGON (((-60.09941101074219 -3.52618050...   
Fam_98 538  MULTIPOLYGON (((-60.04537582397461 -2.84346532...   
       554  MULTIPOLYGON (((-60.06333923339844 -2.48413825...   

                                                      geom_20  \
Fam_0  0    POLYGON ((-58.37060165405273 -3.97397780418396...   
       10   POLYGON ((-58.28065872192383 -3.91993093490600...   
Fam_1  1    POLYGON ((-59.25312805175781 -3.86726546287536...   
       11   POLYGON ((-59.30713653564453 -3.88527107238769...   
Fam_10 20   POLYGON ((-58.55130767822266 -3.57904767990112...   
...                                                       ...   
Fam_97 533  POLYGON ((-59.90138626098633 -3.74177789688110...   
       548  POLYGON ((-59.9013557434082 -4.065171241760254...   
       566  POLYGON ((-59.9734001159668 -3.975343704223633...   
Fam_98 538  POLYGON ((-60.04537582397461 -2.84346532821655...   
       554  POLYGON ((-60.20726776123047 -2.82547879219055...   

                                                      geom_35  \
Fam_0  0    POLYGON ((-58.35262298583984 -3.95598340034484...   
       10                                                None   
Fam_1  1    POLYGON ((-59.23513793945312 -3.84928584098815...   
       11                                                None   
Fam_10 20   POLYGON ((-58.47904205322266 -3.74061918258667...   
...                                                       ...   
Fam_97 533  MULTIPOLYGON (((-59.91938400268555 -3.84957718...   
       548  MULTIPOLYGON (((-59.97340393066406 -3.84957957...   
       566  POLYGON ((-59.99140930175781 -3.95737767219543...   
Fam_98 538  MULTIPOLYGON (((-60.15330123901367 -2.82548832...   
       554  MULTIPOLYGON (((-60.13528442382812 -2.59192895...   

                                                      geom_40  dsize_20  \
Fam_0  0                                                 None     

In [30]:
#### SAVE PICKLE
fam_df.to_pickle('tracking.pkl')

In [ ]:
fam_df[['nc_file','cluster_file']]

In [ ]:


# ## PRIMEIRA LINHA DO NOVO FRAME
# inicial = track_df.iloc[[0]][['nc_file','cluster_file']]

# cluster_matrix = open_cluster(PATH,inicial['cluster_file'].values[0])[:,:,0]
# cluster_matrix[cluster_matrix == 0] = np.nan

In [ ]:
# ### VISUALIZATION
# plt.imshow(cluster_matrix)

In [ ]:
# ### SELECT X Y FROM 'id_t'
# x,y = np.where(cluster_matrix[:,:] == 2)

In [ ]:
# ## READ DBZ MATRIX FROM NETCDF
# dbz_matrix = open_file(inicial['nc_file'].values[0])
# # dbz_matrix[dbz_matrix < THRESHOLD[0]] = np.nan

In [ ]:
# ## VISUALIZATION DBZ MATRIX
# plt.imshow(dbz_matrix)

In [ ]:
# ## VISUALIZATION
# track_df.iloc[[0]][['timestamp','uid','id_t','global_mean_threshold']]

In [ ]:
# print('Tamanho do cluster',len(dbz_matrix[x,y]))

In [ ]:
# dbz_list = dbz_matrix[x,y]

# mmm6 = np.mean(10**(dbz_list/10))
# dbz_mean = (10*np.log10(mmm6))
# print('Resultado do código ->',dbz_mean,' dBZ')

In [ ]:
# mean_mmh =  np.mean((10**(dbz_list/10))/174.8)**(1/1.56)
# mean_reflect = 10*np.log10(200*(mean_mmh**1.6))
# mean_reflect = (mean_reflect)

# mean_reflect

In [ ]:
#         ### COMPARE FLAG AND NO FLAG
#         no_flag_geoms = df_compare.loc[df_compare.timestamp == row.timestamp]
#         for no,rowno in no_flag_geoms.iterrows():
#             if rowno.geometry_t != '-1':
#                 no_geom = shapely.wkt.loads(rowno.geometry_t)
#                 actual_geom = shapely.wkt.loads(row['geometry_t'])
#                 if actual_geom.equals(no_geom):
#                     if row.velm != rowno.velm:
#                         for t in range(len(threshold)):
#                             print(row.velm,rowno.velm, '---- FAR: ',row.FAR_20,' -- FAR_NO',rowno.FAR_20)
#                             df.loc[i,'total_size_'+str(threshold[t])] = len(x)